# Setup

In [ ]:
##install Falcontune and Falcon-7B model
!git clone https://github.com/rmihaylov/falcontune
!wget https://huggingface.co/TheBloke/falcon-7b-instruct-GPTQ/resolve/main/gptq_model-4bit-64g.safetensors

In [ ]:
#install its dependencies
!cd falcontune && pip install -r requirements.txt
!cd falcontune && python setup.py install

# Fine-tune Falcon-7B on toy dataset

In [ ]:
#Get toy dataset
!wget https://github.com/gururise/AlpacaDataCleaned/raw/main/alpaca_data_cleaned.json

In [ ]:
falcontune finetune \
    --model=falcon-7b-instruct-4bit \ #Specify the Falcon model
    --weights=./gptq_model-4bit-64g.safetensors \ #Specify model weights
    --dataset=./alpaca_data_cleaned.json \ #choose your own dataset
    --data_type=alpaca \ 
    --lora_out_dir=./falcon-7b-instruct-4bit-alpaca/ \ #Specify output directory
    --mbatch_size=1 \ 
    --batch_size=2 \ #
    --epochs=3 \ #Specify number of epochs
    --lr=3e-4 \ #Specify learning rate
    --cutoff_len=256 \ 
    --lora_r=8 \ #Specify LoRA parameters
    --lora_alpha=16 \
    --lora_dropout=0.05 \
    --warmup_steps=5 \ 
    --save_steps=50 \ #Specify how often to save model checkpoints
    --save_total_limit=3 \ #Specify how many checkpoints to save
    --logging_steps=5 \ #Specify how often to log
    --target_modules='["query_key_value"]' \ 
    --backend=triton

# Load

In [ ]:
!falcontune generate \
    --interactive \  #Specify interactive mode
    --model falcon-40b-instruct-4bit \ #Specify the Falcon model
    --weights ./gptq_model-4bit--1g.safetensors \
    --lora_apply_dir falcon-40b-instruct-4bit-alpaca/ \
    --max_new_tokens 50 \ #Specify how many tokens to generate
    --use_cache \ #Specify whether to use cache
    --do_sample \ 
    --instruction "How to prepare pasta?" \ #Specify the query
    --backend triton 